In [1]:
import re
import regex
import json
import emoji
import emojis
import humanize
import calendar
import datetime
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from dateutil.relativedelta import relativedelta
from collections import Counter

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
import seaborn as sns
sns.set()

import PIL.Image
from IPython.display import Image
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [3]:
import pickle

from wordcloud import WordCloud
from wordcloud import ImageColorGenerator

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from nltk import pos_tag
from nltk.util import ngrams 
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

import pyLDAvis.gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel

import spacy
from spacy.lang.en.stop_words import STOP_WORDS as stopwords_en_spacy
from spacy.lang.es.stop_words import STOP_WORDS as stopwords_sp_spacy

/opt/anaconda3/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.



### Load my tweets provided by Twitter itself

In [7]:
with open('../data/tweet.js') as f:
    file = f.read()
    file = file[file.find("["):]
    
data = json.loads(file)
print(f"{len(data):,.0f} tweets")
df = json_normalize(data)

16,534 tweets


### Data cleaning

In [8]:
# Remove RTs

df['RTs'] = df['tweet.full_text'].apply(lambda text: text.startswith("RT @"))
RTs = df[df['RTs']]['tweet.full_text'].apply(lambda text: text.split(":")[0]).values

print(df['RTs'].value_counts())
df = df[~df['RTs']]

False    10577
True      5957
Name: RTs, dtype: int64


In [9]:
df.drop(['tweet.id_str',
         'tweet.retweeted', 
         'tweet.favorited',
         'tweet.truncated', 
         'tweet.source', 
         'tweet.possibly_sensitive', 
         'tweet.entities.symbols',
         'tweet.extended_entities.media',
         'tweet.display_text_range', 
         'tweet.in_reply_to_status_id',
         'tweet.in_reply_to_status_id_str', 
         'tweet.in_reply_to_user_id', 
         'tweet.in_reply_to_user_id_str',
         'tweet.coordinates.type', 
         'tweet.coordinates.coordinates', 
         'tweet.geo.type', 
         'tweet.geo.coordinates',
         'RTs'
          ], axis=1, inplace=True)
df.shape

(10577, 11)

In [10]:
def get_urls(obj):
    urls = []
    if len(obj)==0:
        return urls
    for item in obj:
        urls.append(item['expanded_url'])
    return urls

df['urls'] = df['tweet.entities.urls'].apply(lambda obj: get_urls(obj))
df.drop(['tweet.entities.urls'], axis=1, inplace=True)

In [11]:
def get_mentions(obj):
    mentions = []
    if len(obj)==0:
        return mentions
    for item in obj:
        mentions.append(item['screen_name'])
    return mentions

df['mentions'] = df['tweet.entities.user_mentions'].apply(lambda obj: get_mentions(obj))
df.drop(['tweet.entities.user_mentions'], axis=1, inplace=True)

def remove_replies(mentions, replied_to):
    try:
        mentions.remove(replied_to)
        return mentions
    except:
        return mentions

df['mentions'] = df.apply(lambda x: remove_replies(x.mentions, x['tweet.in_reply_to_screen_name']), axis=1)

In [12]:
def get_hashtags(obj):
    hashtags = []
    if len(obj)==0:
        return hashtags
    for item in obj:
        hashtags.append(item['text'])
    return hashtags

df['hashtags'] = df['tweet.entities.hashtags'].apply(lambda obj: get_hashtags(obj))
df.drop(['tweet.entities.hashtags'], axis=1, inplace=True)

In [13]:
def get_media_urls(obj):
    media_urls = []
    try:
        if len(obj)==0:
            return media_urls
    except:
        return media_urls
    for item in obj:
        media_urls.append(item['expanded_url'])
    return media_urls

df['media.urls'] = df['tweet.entities.media'].apply(lambda obj: get_media_urls(obj))
df.drop(['tweet.entities.media'], axis=1, inplace=True)

In [14]:
df = df[['tweet.id',
         'tweet.created_at',
         'tweet.full_text',
         'tweet.lang',
         'tweet.retweet_count',
         'tweet.favorite_count',
         'mentions',
         'tweet.in_reply_to_screen_name',
         'hashtags',
         'urls',
         'media.urls'
         ]]

df.columns = ['id', 'date', 'text', 'lang',
              'retweets', 'favorites', 'mentions', 'replied_to',
              'hashtags', 'urls', 'media_urls']

### Data manipulation

In [15]:
df['date'] = pd.to_datetime(df.date)
df['day'] = df.date.dt.date
df['YYYYMM'] = df.date.dt.to_period('M')
df['year'] = df.date.dt.year
df['dow'] = df.date.dt.day_name()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1269: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [16]:
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

df['emojis'] = df['text'].apply(lambda text: get_emojis(text))

In [17]:
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(text):
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text

df['text_clean'] = df['text'].apply(lambda text: text_clean(text))

<>:1: DeprecationWarning:

invalid escape sequence \s

<>:2: DeprecationWarning:

invalid escape sequence \s

<>:1: DeprecationWarning:

invalid escape sequence \s

<>:2: DeprecationWarning:

invalid escape sequence \s

<>:1: DeprecationWarning:

invalid escape sequence \s

<>:2: DeprecationWarning:

invalid escape sequence \s

<ipython-input-17-01c43c42085e>:1: DeprecationWarning:

invalid escape sequence \s

<ipython-input-17-01c43c42085e>:2: DeprecationWarning:

invalid escape sequence \s



In [35]:
df.sample(3)

,id,date,text,lang,retweets,favorites,mentions,replied_to,hashtags,urls,media_urls,day,YYYYMM,year,dow,emojis,text_clean,id_url
13315,417361401204977664,2013-12-29 18:28:06+00:00,In science every answer raises more questions.,en,0,0,[],NaN,[],[],[],2013-12-29,2013-12,2013,Sunday,[],In science every answer raises more questions.,https://mobile.twitter.com/user/status/4173614...
3856,830017078711218176,2017-02-10 11:34:27+00:00,"Шесть утра, и я смотрю через окно. Каждый день...",ru,0,0,[],NaN,[],[],[https://twitter.com/vivmarquez/status/8300170...,2017-02-10,2017-02,2017,Friday,[],"Шесть утра, и я смотрю через окно. Каждый день...",https://mobile.twitter.com/user/status/8300170...
13072,1260293929615925255,2020-05-12 19:41:02+00:00,Les traigo un buen descubrimiento musical: Mar...,en,0,0,[YouTube],NaN,[],[https://youtu.be/3ApUE9wZ0yA],[],2020-05-12,2020-05,2020,Tuesday,[],Les traigo un buen descubrimiento musical: Mar...,https://mobile.twitter.com/user/status/1260293...


In [19]:
df['id_url'] = df['id'].apply(lambda text: "https://mobile.twitter.com/user/status/"+text)

### EDA

In [36]:
my_color = "#ff00a7"

In [37]:
print("Time on Twitter with this account:")
print(f"Since {humanize.naturaldate(df.date.min())}")
relativedelta(df.date.max(), df.date.min())

Time on Twitter with this account:
Since Jun 04 2011


relativedelta(years=+9, days=+15, minutes=+42, seconds=+35)

In [38]:
# Plot Day
temp_day = pd.DataFrame(df.day.value_counts())
temp_day.reset_index(inplace=True)
temp_day.columns = ['day', 'tweet_count']
temp_day.sort_values(by=['day'], inplace=True)
temp_day.reset_index(inplace=True, drop=True)
idx = pd.date_range(temp_day.day.min(), temp_day.day.max())
temp_day.index = pd.DatetimeIndex(temp_day.day)
temp_day = temp_day.reindex(idx, fill_value=0)
temp_day.day = temp_day.index
temp_day.reset_index(drop=True, inplace=True)


trace_day = go.Scatter(
                    x=temp_day.day.astype(str).values,
                    y=temp_day.tweet_count.values,
                    text = [f"{humanize.naturaldate(day)}: {count} tweets" for day,count in zip(temp_day.day.dt.date.values,temp_day.tweet_count.values)],
                    hoverinfo='text',
                    mode='lines', 
                    line = {
                        'color': my_color,
                        'width': 1.2
                    },
                    visible=False,
                    name="Day"
                )

# Plot Month
temp_month = temp_day
temp_month.index = pd.DatetimeIndex(temp_month.day)
temp_month = temp_month.groupby(pd.Grouper(freq='M')).sum().reset_index()
idx = pd.date_range(temp_month.day.min(), temp_month.day.max())
temp_month.index = pd.DatetimeIndex(temp_month.day)
temp_month = temp_month.reindex(idx, fill_value=0)
temp_month.day = temp_month.index
temp_month.reset_index(drop=True, inplace=True)
temp_month['YYYYMM'] = temp_month['day'].dt.to_period('M')
temp_month.drop_duplicates(subset=['YYYYMM'], keep='last', inplace=True)


trace_month = go.Scatter(
                    x=temp_month.day.astype('datetime64[M]').astype(str).values,
                    y=temp_month.tweet_count.values,
                    text = [f"{humanize.naturaldate(month)[:3]+humanize.naturaldate(month)[6:]}: {count} tweets" for month,count in zip(temp_month.YYYYMM.values,temp_month.tweet_count.values)],
                    hoverinfo='text',
                    mode='lines', 
                    line = {
                        'color': my_color,
                        'width': 1.2
                    },
                    visible=False,
                    name="Month"
                ) 

# Plot year
temp_year = temp_day
temp_year.index = pd.DatetimeIndex(temp_year.day)
temp_year = temp_year.groupby(pd.Grouper(freq='Y')).sum().reset_index()
temp_year['year'] = temp_year.day.dt.year
temp_year

trace_year = go.Scatter(
                    x=temp_year.day.astype('datetime64[Y]').astype(str).values,
                    y=temp_year.tweet_count.values,
                    text = [f"Year {year}: {count:,.0f} tweets" for year,count in zip(temp_year.year.values,temp_year.tweet_count.values)],
                    hoverinfo='text',
                    mode='lines+markers', 
                    line = {
                        'color': my_color,
                        'width': 1.2
                    },
                    visible=True,
                    name="Year"
                ) 


# Menus
updatemenus = list([
    dict(
         active=0,
         buttons=list([
            dict(label = 'Year',
                 method = 'update',
                 args = [{'visible': [True, False, False]},
                         {'title': 'Number of Tweets per Year'}]),
            dict(label = 'Month',
                 method = 'update',
                 args = [{'visible': [False, True, False]},
                         {'title': 'Number of Tweets per Month'}]),
            dict(label = 'Day',
                 method = 'update',
                 args = [{'visible': [False, False, True]},
                         {'title': 'Number of Tweets per Day'}]),
             ]),
        
    )
])

# Layout
layout = go.Layout(title="Number of Tweets -- Pick a scale",
                   updatemenus=updatemenus,
                  )

fig = go.Figure(data=[trace_year, trace_month, trace_day], layout=layout)
iplot(fig)

In [48]:
pio.write_html(fig, file='../plot/Plot1.html', auto_open=False)

FileNotFoundError: [Errno 2] No such file or directory: '../plot/Plot1.html'

In [39]:
temp = pd.DataFrame(df.groupby('day').count()['date'])
temp.reset_index(inplace=True)
temp.columns = ['date', 'tweet_count']
temp['date_human'] = temp['date'].apply(lambda value: humanize.naturaldate(value))
print("Days I tweeted the most:")
temp[['date_human', 'tweet_count']].sort_values('tweet_count', ascending=False)[:5].reset_index(drop=True)

Days I tweeted the most:


,date_human,tweet_count
0,Jan 24 2013,62
1,Jul 02 2017,55
2,Mar 14 2015,48
3,Jan 26 2015,46
4,Dec 20 2012,35


In [ ]:
temp['date'] = pd.to_datetime(temp['date'])
temp = temp[temp['date'].dt.year > 2012]
temp['diff'] = temp['date'].diff().dt.days
temp['date'] = temp['date'].apply(lambda value: humanize.naturaldate(value))
print("Longest streaks without tweeting:")
temp[['date_human', 'diff']].sort_values('diff', ascending=False)[:5].reset_index(drop=True)

In [ ]:
temp = pd.DataFrame(df['lang'].value_counts()).reset_index()
temp.columns = ['language', 'tweet_count']

lang_dict = {'es': 'Spanish',
             'en': 'English',
             'pt': 'Portuguese',
             'ru': 'Russian',
             'fr': 'French',
             'ja': 'Japanese'}

# Others will include: mixed, one-word, one-emoji, missclasified, and otherss

temp['language'] = temp['language'].apply(lambda lg: lang_dict[lg] if lg in lang_dict.keys() else 'Others')
temp = temp.groupby('language').sum().sort_values('tweet_count', ascending=False).reset_index()

# Plot
trace = go.Bar(
                x=temp.language.values,
                y=temp.tweet_count.values,
                opacity=0.6,
                marker=dict(color=my_color,
                            line=dict(color=my_color,width=1.5,)
                           ),
                )

layout = go.Layout(title='Number of tweets per language<br> ALL TIME',
                   yaxis=dict(tickformat=','))

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [ ]:
traces = []

lang_color_dict = {'Spanish': "#f4ce26",
                   'English': "#2093dd",
                   'Portuguese': "#037f3e",
                   'Russian': "#2135a3",
                   'French': "#dd3228",
                   'Japanese': "#b666de"}

temp = df[['lang', 'YYYYMM']]
temp['language'] = temp['lang'].apply(lambda lg: lang_dict[lg] if lg in lang_dict.keys() else 'Others')
temp = temp[temp['language']!='Others']
temp['YYYYMM'] = temp['YYYYMM'].dt.strftime('%Y-%m')
temp = temp.groupby(['language', 'YYYYMM'])['language'].count()
temp = pd.DataFrame(temp / temp.groupby(level=1).sum())
temp.columns = ['pct']
temp.reset_index(inplace=True)

for lang in temp['language'].unique():
    trace= go.Scatter(
                        x=temp[temp['language']==lang].YYYYMM.values,
                        y=temp[temp['language']==lang].pct.values,
                        mode='lines', 
                        line = {
                            'color': lang_color_dict[lang],
                            'width': 1.2
                        },
                        name=lang
                    ) 

    traces.append(trace)
    
layout = go.Layout(title='Percentage of tweets per language',
                   yaxis=dict(tickformat=',.0%'))

fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [ ]:
pd.set_option('display.max_colwidth', -1)
df.retweets = pd.to_numeric(df.retweets)
print(f"Max number of retweets: {df.retweets.max()}")

df[['day', 'text_clean', 'retweets', 'favorites', 'id_url']].sort_values('retweets', ascending=False).head(5).reset_index(drop=True)

In [ ]:
pd.set_option('display.max_colwidth', -1)
df.favorites = pd.to_numeric(df.favorites)
print(f"Max number of favorites: {df.favorites.max()}")

df[['day', 'text_clean', 'retweets', 'favorites', 'id_url']].sort_values('favorites', ascending=False).head(5).reset_index(drop=True)

In [ ]:
Counter(sum(df['mentions'].values,[])).most_common(10)

In [ ]:
df['replied_to'].value_counts()[:11]

In [ ]:
Counter(RTs).most_common(10)

In [ ]:
print(f"Total tweets: 16,534")
print(f"From which...")
print(f"5,957 are re-tweets")
print(f"{df['replied_to'].isna().value_counts()[False]:,.0f} are replies")
print(f"{sum(df['urls'].apply(lambda value: True if len(value)!=0 else False) | df['media_urls'].apply(lambda value: True if len(value)!=0 else False)):,.0f} have media")

In [ ]:
Counter(sum(df['urls'].values,[])).most_common(5)

 HASHTAGS (Tableau)

In [ ]:
temp_year = []

for year in list(range(2011,2021)):
    temp = pd.DataFrame(Counter(sum(df[df['year']==year]['hashtags'].values,[])).most_common(30))
    temp['year'] = year
    temp_year.append(temp)
    
temp_year = pd.concat(temp_year)
temp_year.columns = ['hashtag', 'tweet_count', 'year']

temp_year.to_csv("hashtag.csv")

In [ ]:
vader = SentimentIntensityAnalyzer() 

def transform_emoji(es):
    if len(es)==0:
        return []
    
    result = []
    for e in es:
        e_string = emoji.demojize(e)[1:-1].replace("_"," ")
        e_list = [(emoji, name) for emoji, name in vader.emojis.items() if e_string==name]
        if len(e_list)==0:
            result.append(e)
            continue
        result.append(e_list[0][0])
    return result

df['emojis'] = df['emojis'].apply(lambda es: transform_emoji(es))

In [ ]:
# All time
print(f"{sum(list(Counter(sum(df['emojis'].values,[])).values())):,.0f} emojis")
print(f"{len(Counter(sum(df['emojis'].values,[])).most_common())} different ones\n\n")
      
for i, em in enumerate(Counter(sum(df['emojis'].values,[])).most_common()):
    e = em[0]
    print(e)
    print(f"Rank #{i+1}")
    print(f"Count: {em[1]}")
    try:
        print(f"Sentiment: {vader.polarity_scores(e)['compound']}")
        print(f"Category: {emojis.db.get_emoji_by_code(e).category}")
        print()
    except:
        print("NA************************************************")
        print()

In [ ]:
# Per year
temp_emoji = []

for year in list(range(2011,2021)):
    temp = pd.DataFrame(Counter(sum(df[df['year']==year]['emojis'].values,[])).most_common(10))
    temp['year'] = year
    temp_emoji.append(temp)
    
temp_emoji = pd.concat(temp_emoji)
temp_emoji.columns = ['emojis', 'tweet_count', 'year']

temp_emoji = temp_emoji.pivot(index='emojis', columns='year').reset_index()
temp_emoji.columns = ['emojis'] + list(temp_emoji.columns.get_level_values(1)[1:])

# I hard-coded the colors because all emoji libraries out there suck!
gray = '#d2d2d0' # Others
green = '#9ae397' # Nature
yellow = '#fbf690' # Happy
orange = '#ffbe6d' # Travel
pink = '#ff6dbe' # Hearts
blue = '#6dbeff' # Sad

emojis_colors = [gray, green, green, yellow, pink, gray, orange, yellow, yellow, green, 
                 pink, orange, orange, gray, yellow, yellow, yellow, pink, pink, pink, 
                 pink, yellow, yellow, yellow, yellow, yellow, yellow, yellow, yellow, yellow, 
                 blue, blue, blue, blue, blue, yellow, blue, yellow, blue]

temp_emoji['colors'] = emojis_colors

In [ ]:
beg = 2014
listOfFrames = []
for year in range(beg,2021):
    temp = temp_emoji[['emojis', year, 'colors']]
    temp = temp.sort_values(year, ascending=False).dropna()
    
    trace = go.Bar(
                x=temp.emojis.values,
                y=temp[year].values,
                textposition = "outside",
                texttemplate = "%{x}<br>%{y}",
                textfont = {'size': [14]*10},
                hoverinfo='none', 
                opacity=1,
                marker=dict(color=temp.colors.values,
                           ),
                cliponaxis = False
                )

    layout = go.Layout(title=f'Emojis in {year}',
                       xaxis = {"range":[-1,10],"autorange": False, "showline":False,"tickangle":-90, "visible":False},
                       yaxis = {"range":[-1,200],"autorange": False, "showline":False, "visible":False},
                       plot_bgcolor= '#FFFFFF')
    listOfFrames.append(go.Frame(data=[trace], layout=layout))
    
frames=list(listOfFrames)

year = beg
temp = temp_emoji[['emojis', year, 'colors']]
temp = temp.sort_values(year, ascending=False).dropna()

# Plot
trace = go.Bar(
                x=temp.emojis.values,
                y=temp[year].values,
                textposition = "outside",
                texttemplate = "%{x}<br>%{y}",
                textfont = {'size': [14]*10},
                hoverinfo='none', 
                opacity=1,
                marker=dict(color=temp.colors.values,
                           ),
                cliponaxis = False
                )

layout = go.Layout(title=f'Emojis in {year}',
                   xaxis = {"range":[-1,10],"autorange": False, "showline":False,"tickangle":-90, "visible":False},
                   yaxis = {"range":[-1,200],"autorange": False, "showline":False, "visible":False},
                   plot_bgcolor= '#FFFFFF',
                   updatemenus=[dict(type="buttons",
                                     buttons=[dict(label="▶️",
                                                   method="animate",
                                                   args=[None, {"frame": {"duration": 800, "redraw": True},
                                                                "fromcurrent": True}]),
                                              {
                "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "⏸️",
                "method": "animate"
            }
                                             ]
                                    )
                               ]
                  )

fig = go.Figure(data=[trace], layout=layout, frames=frames)

fig.show()

In [ ]:
# Per month
temp_emoji = []

for ym in sorted(df.YYYYMM.unique()):
    temp = pd.DataFrame(Counter(sum(df[df['YYYYMM']==ym]['emojis'].values,[])).most_common(5))
    temp['YYYYMM'] = ym
    if temp.shape[0] != 0:
        temp_emoji.append(temp)
    
temp_emoji = pd.concat(temp_emoji)
temp_emoji.columns = ['emojis', 'tweet_count', 'YYYYMM']

temp_emoji = temp_emoji.pivot(index='emojis', columns='YYYYMM').reset_index()
temp_emoji.columns = ['emojis'] + list(temp_emoji.columns.get_level_values(1)[1:])

# I hard-coded the colors because all emoji libraries out there suck!
gray = '#d2d2d0' # Others
green = '#9ae397' # Nature
yellow = '#fbf690' # Happy
orange = '#ffbe6d' # Travel
pink = '#ff6dbe' # Hearts
blue = '#6dbeff' # Sad
purple = "#be6dff" # Celebration

emojis_colors = [gray, green, green, gray, blue, yellow, pink, gray, orange, yellow, 
                 yellow, green, green, pink, orange, orange, orange, orange, orange, green, 
                 green, green, green, purple, purple, purple, purple, purple, gray, gray, 
                 green, gray, gray, yellow, yellow, yellow, gray, pink, pink, pink, 
                 pink, pink, gray, yellow, gray, gray, yellow, yellow, yellow, yellow, 
                 yellow, yellow, yellow, yellow, yellow, blue, blue, blue, blue, yellow, 
                 blue, blue, blue, blue, blue, blue, blue, blue, blue, blue, 
                 yellow, blue, yellow, blue, yellow, yellow, yellow, yellow, yellow, yellow, 
                 gray, blue, gray, yellow, yellow, yellow, blue, yellow, blue, green]

temp_emoji['colors'] = emojis_colors

In [ ]:
beg = temp_emoji.columns[13]
listOfFrames = []
for year in temp_emoji.columns[1:-1]:
    temp = temp_emoji[['emojis', year, 'colors']]
    temp = temp.sort_values(year, ascending=True).dropna()
    if temp.shape[0]!=5:
        continue
    
    trace = go.Bar(
                y=temp.emojis.values,
                x=temp[year].values,
                orientation='h',
                textposition = "outside",
                texttemplate = "%{x}",
                textfont = {'size': [10]*10},
                hoverinfo='none', 
                opacity=1,
                marker=dict(color=temp.colors.values,
                           ),
                cliponaxis = False
                )

    layout = go.Layout(title=f'Emojis in {calendar.month_name[humanize.naturaltime(year).month]} {humanize.naturaltime(year).year}',
                       yaxis = {"range":[-1,5],"autorange": False, "showline":False, "visible":True},
                       xaxis = {"range":[-1,150],"autorange": False, "showline":False, "visible":False},
                       plot_bgcolor= '#FFFFFF')
    listOfFrames.append(go.Frame(data=[trace], layout=layout))
    
frames=list(listOfFrames)

year = beg
temp = temp_emoji[['emojis', year, 'colors']]
temp = temp.sort_values(year, ascending=True).dropna()

# Plot
trace = go.Bar(
                y=temp.emojis.values,
                x=temp[year].values,
                orientation='h',
                textposition = "outside",
                texttemplate = "%{x}",
                textfont = {'size': [10]*10},
                hoverinfo='none', 
                opacity=1,
                marker=dict(color=temp.colors.values,
                           ),
                cliponaxis = False
                )

layout = go.Layout(title=f'Emojis in {calendar.month_name[humanize.naturaltime(year).month]} {humanize.naturaltime(year).year}',
                   yaxis = {"range":[-1,5],"autorange": False, "showline":False, "visible":True},
                   xaxis = {"range":[-1,150],"autorange": False, "showline":False, "visible":False},
                   plot_bgcolor= '#FFFFFF',
                   updatemenus=[dict(type="buttons",
                                     buttons=[dict(label="▶️",
                                                   method="animate",
                                                   args=[None, {"frame": {"duration": 1200, "redraw": True},
                                                                "fromcurrent": True}]),
                                              {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "⏸️",
                "method": "animate"
            }
                                             ]
                                    )
                               ]
                  )

fig = go.Figure(data=[trace], layout=layout, frames=frames)

fig.show()

In [ ]:
all_text = re.sub("[^'’ \w]|_"," ", " ".join(df['text_clean'].values))
words = [word for word in all_text.split() if len(word)>2]

print(f"Total characters written: {len(all_text):,.0f}")
print(f"Number of words: ± {len(words):,.0f}")
print(f"Number of unique words: ± {len(set(words)):,.0f}")

In [ ]:
stopwords_en = stopwords.words('english')
stopwords_sp = stopwords.words('spanish')
sw = set(stopwords_en + stopwords_sp + list(stopwords_en_spacy) + list(stopwords_sp_spacy))
sw.add("can't")
sw.add("like")


words_long2 = [word for word in words if len(word)>2]
words_long3 = [word for word in words if len(word)>3]
words_long4 = [word for word in words if len(word)>4]

words_sw2 = [word for word in words_long2 if word.lower() not in sw]
words_sw3 = [word for word in words_long3 if word.lower() not in sw]
words_sw4 = [word for word in words_long4 if word.lower() not in sw]

In [ ]:
print("Top words used:\n")
Counter(words_sw4).most_common()[:10]

In [ ]:
n_gram = 2
Counter(ngrams(words_sw2, n_gram)).most_common(50)

In [ ]:
words = all_text
wordcloud = WordCloud(width=1600, height=800, max_font_size=200, stopwords=sw,
                      background_color='white', colormap='Dark2').generate(words)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show();

In [ ]:
def most_frequent_words_visualization(df, col='text', title="", color_palette="Dark2"):
        '''
        input: data frame with tweets.
        output: word cloud with most frequent words from column "text", i.e., the tweets themselves.
        '''
        words = " ".join(sum(df[col].values,[]))
        wordcloud = WordCloud(width=1600, height=800, max_font_size=200, 
                              background_color='white', colormap=color_palette).generate(words)
        plt.figure(figsize=(15,10))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(title)
        plt.axis("off")
        plt.show()

In [ ]:
def processed_text(text_clean):
    text_clean = text_clean.replace("’","'")
    text_clean = re.sub("[^'’ \w]|_"," ", text_clean)
    words = [word for word in text_clean.split() if (len(word)>2) and (word.lower() not in sw)]
    return words

df['processed_text'] = df['text_clean'].apply(lambda tc: processed_text(tc))

In [ ]:
most_frequent_words_visualization(df[df['lang']=='pt'], 'processed_text', "Portuguese")

In [ ]:
most_frequent_words_visualization(df[df['lang']=='ru'], 'processed_text', "Russian")

In [ ]:
most_frequent_words_visualization(df[df['lang']=='fr'], 'processed_text', "French")

In [ ]:
most_frequent_words_visualization(df[df['lang'].isin(['ru', 'pt', 'fr'])], 'processed_text', "RU-PT-FR")

In [ ]:
for year in sorted(df.year.unique()):
    mask = np.array(PIL.Image.open("twitter.png"))
    words = " ".join(sum(df[df['year']==year]['processed_text'].values,[]))
    wc = WordCloud(background_color="white", mode="RGBA", max_words=500, max_font_size=200, mask=mask).generate(words)
    image_colors = ImageColorGenerator(mask)
    plt.figure(figsize=(15,15));
    plt.title(year);
    plt.imshow(wc.recolor(color_func=image_colors), interpolation='bilinear');
    plt.axis("off");
    plt.show();

# Topic Modeling

In [ ]:
englishStemmer=SnowballStemmer("english")
spanishStemmer=SnowballStemmer("spanish")

df_english = df[df['lang']=='en']
df_spanish = df[df['lang']=='es']

In [ ]:
def get_doc_top_n(text_processed_lemma, n):
    d = dictionary.doc2bow(text_processed_lemma)
    topics = ldamodel.get_document_topics(d)
    try:
        return topics[n][1]
    except:
        return None

In [ ]:
def text_processed_lemma(doc, stemmer):
    doc = sum([re.sub("[^\w]|_"," ",word).split() for word in doc],[])
    doc = [word for word in doc if len(word)>2 and word.lower() not in sw]
    if "math" in doc or "maths" in doc:
        doc = ["mathematics" if (word=="math" or word=="maths") else word for word in doc]
    pt = pos_tag(doc)
    doc = [word for word, typ in pt if typ[0] in ['N']]
    doc_stem = [stemmer.stem(w) for w in doc if len(w)>3]
    if len(doc_stem)<5:
        return None
    return doc_stem
    '''
    doc_bg = []
    for key,value in Counter(ngrams([word.lower() for word in doc],2)).items():
        for i in range(value):
            doc_bg.append(" ".join(key))
    doc = doc_stem + doc_bg
    return doc
    '''

In [ ]:
df_english['text_processed_lemma'] = df_english['processed_text'].apply(lambda doc: text_processed_lemma(doc, englishStemmer))
df_english = df_english.dropna(subset=['text_processed_lemma'])
df_english = df_english[df_english.index.isin(df_english['text_processed_lemma'].apply(tuple, 1).drop_duplicates().index)]

df_spanish['text_processed_lemma'] = df_spanish['processed_text'].apply(lambda text: text_processed_lemma(text, spanishStemmer))
df_spanish = df_spanish.dropna(subset=['text_processed_lemma'])
df_spanish = df_spanish[df_spanish.index.isin(df_spanish['text_processed_lemma'].apply(tuple, 1).drop_duplicates().index)]

In [ ]:
dictionary = corpora.Dictionary(list(df_english.text_processed_lemma.values))
dictionary.filter_extremes(no_below=20, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in list(df_english.text_processed_lemma.values)]

NUM_TOPICS = 2
ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30, alpha=0.01, eta=0.01)
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)
    print()
    
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

# pyLDAvis.save_html(lda_display, 'lda.html')

In [ ]:
for t in range(0,NUM_TOPICS):
    top_name = f"topic_{t}"
    df_english[top_name] = df_english['text_processed_lemma'].apply(lambda doc: get_doc_top_n(doc, t))

In [ ]:
for t in range(0,NUM_TOPICS):
    print(f"****************************** TOPIC {t} ******************************")
    topic = f"topic_{t}"
    for i, tweet in enumerate(df_english.sort_values(topic, ascending=False)['text_clean'].head()):
        print(f"Tweet #{i+1}")
        print(tweet)
        print()
    print()

In [ ]:
dictionary = corpora.Dictionary(list(df_spanish.text_processed_lemma.values))
dictionary.filter_extremes(no_below=20, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in list(df_spanish.text_processed_lemma.values)]

NUM_TOPICS = 3
ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30, alpha=0.01, eta=0.01)
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)
    print()
    
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

# pyLDAvis.save_html(lda_display, 'lda.html')

In [ ]:
for t in range(0,NUM_TOPICS):
    top_name = f"topic_{t}"
    df_spanish[top_name] = df_spanish['text_processed_lemma'].apply(lambda doc: get_doc_top_n(doc, t))

In [ ]:
for t in range(0,NUM_TOPICS):
    print(f"****************************** TOPIC {t} ******************************")
    topic = f"topic_{t}"
    for i, tweet in enumerate(df_spanish.sort_values(topic, ascending=False)['text_clean'].head()):
        print(f"Tweet #{i+1}")
        print(tweet)
        print()
    print()

# K Means

In [ ]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def text_processed_lemma_en(text_clean):
    doc = processed_text(text_clean)
    stemmer = englishStemmer
    doc = sum([re.sub("[^\w]|_"," ",word).split() for word in doc],[])
    doc = [word for word in doc if len(word)>2 and word.lower() not in sw]
    if "math" in doc or "maths" in doc:
        doc = ["mathematics" if (word=="math" or word=="maths") else word for word in doc]
    pt = pos_tag(doc)
    doc = [word for word, typ in pt if typ[0] in ['N', 'V']]
    doc_stem = [stemmer.stem(w) for w in doc if len(w)>3]
    #if len(doc_stem)<5:
    #    return None
    return doc_stem
    '''
    doc_bg = []
    for key,value in Counter(ngrams([word.lower() for word in doc],2)).items():
        for i in range(value):
            doc_bg.append(" ".join(key))
    doc = doc_stem + doc_bg
    return doc
    '''

In [ ]:
token_vectorizer = CountVectorizer(tokenizer = text_processed_lemma_en, min_df=10, stop_words=sw, ngram_range=(1, 2))
features = token_vectorizer.fit_transform(df_english.text_clean)

In [ ]:
ks = []
kinertia = []

for k in range(1,20):
    kmeans = KMeans(n_clusters=k).fit(features)
    ks.append(k)
    kinertia.append(kmeans.inertia_)
    
plt.plot(ks,kinertia, 'bx-');

In [ ]:
true_k = 17
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(features)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = token_vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print()


In [ ]:
df_english['kmeans'] = model.labels_
df_english.kmeans.value_counts()

In [ ]:
def text_processed_lemma_sp(text_clean):
    doc = processed_text(text_clean)
    stemmer = spanishStemmer
    doc = sum([re.sub("[^\w]|_"," ",word).split() for word in doc],[])
    doc = [word for word in doc if len(word)>2 and word.lower() not in sw]
    if "math" in doc or "maths" in doc:
        doc = ["mathematics" if (word=="math" or word=="maths") else word for word in doc]
    pt = pos_tag(doc)
    doc = [word for word, typ in pt if typ[0] in ['N', 'V']]
    doc_stem = [stemmer.stem(w) for w in doc if len(w)>3]
    #if len(doc_stem)<5:
    #    return None
    return doc_stem
    '''
    doc_bg = []
    for key,value in Counter(ngrams([word.lower() for word in doc],2)).items():
        for i in range(value):
            doc_bg.append(" ".join(key))
    doc = doc_stem + doc_bg
    return doc
    '''

In [ ]:
token_vectorizer = CountVectorizer(tokenizer = text_processed_lemma_sp, min_df=10, stop_words=sw, ngram_range=(1, 2))
features = token_vectorizer.fit_transform(df_spanish.text_clean)

In [ ]:
ks = []
kinertia = []

for k in range(1,20):
    kmeans = KMeans(n_clusters=k).fit(features)
    ks.append(k)
    kinertia.append(kmeans.inertia_)
    
plt.plot(ks,kinertia, 'bx-');

In [ ]:
true_k = 11
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(features)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = token_vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print()


In [ ]:
df_spanish['kmeans'] = model.labels_
df_spanish.kmeans.value_counts()